In [ ]:
import random
import cv2
import numpy as np

In [ ]:
# Reference from https://github.com/swook/GazeML/blob/master/src/datasources/unityeyes.py
augmentation_ranges = {  # (easy, hard)
    'resize': (0.5, 2.0),
    'translation': (0, 50.0),
    'flip':(0,1),
    'rotation': (0, 45),
    'blur': (0.0, 6.0),
    'intensity': (0, 20)
}
blur_type = [
    "mean","gaussian","bilateral","median"
]

In [ ]:
def center_crop(img, o_shape):
    oh, ow = o_shape
    ih, iw = img.shape[0], img.shape[1]
    x = (iw - ow)//2
    y = (ih - oh)//2
    return img[y:y+oh,x:x+ow]

def rotate_bound(image, angle):
    # Grab the dimensions of the image and then determine the center
    (h, w) = image.shape[:2]
    (cX, cY) = (w / 2, h / 2)

    # Grab the rotation matrix (applying the negative of the angle to rotate clockwise)
    # then grab the sine and cosine (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # Compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # Adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # Perform the actual rotation and return the image
    return M, (nH, nW)

def blur_img(img,blur_type,k):
    if k == 0:
        return img
    else:
        k_s = 2*k + 3
        if blur_type == "mean":
            blur = cv2.blur(img,(k_s,k_s))
        elif blur_type == "guassian":
            blur = cv2.GaussianBlur(img,(k_s,k_s),0)
        elif blur_type == "median":
            blur = cv2.medianBlur(img,k_s)
        elif blur_type == "bilateral":
            blur = cv2.bilateralFilter(img,k_s,75,75)
        else:
            blur = img

        return blur

In [ ]:
def aug(img, masks, rotated_rect, o_shape, flag, diff, k_dilate=0):
    ih, iw = img.shape[0], img.shape[1]

    # Augmentation helper values and methods
    random_multipliers = [] # Noise value need to multiply gaussian
    def value_from_type(augmentation_type):
        # Scale to be in range
        easy_value, hard_value = augmentation_ranges[augmentation_type]
        value = (hard_value - easy_value) * diff + easy_value
        value = (np.clip(value, easy_value, hard_value)
                 if easy_value < hard_value
                 else np.clip(value, hard_value, easy_value))
        return value

    def noisy_value_from_type(augmentation_type):
        # Get normal distributed random value
        if len(random_multipliers) == 0:
            random_multipliers.extend(
                    list(np.random.normal(size=(len(augmentation_ranges),))))
        return random_multipliers.pop() * value_from_type(augmentation_type)
    
    # Bluring images using different types of bluring mehtod
    img = blur_img(img, np.random.choice(blur_type),int(np.abs(noisy_value_from_type('blur'))))
    
    # Flip and intensity
    flip = np.random.choice([True, False]) if diff > 0 else False
    if flip:
        img = cv2.flip(img, 1)
        masks = [cv2.flip(m,1) for m in masks]
        rotated_rect[0][0] = iw - rotated_rect[0][0]
        rotated_rect[2] = rotated_rect[2]*-1

    intensity_noise = int(value_from_type('intensity'))
    if intensity_noise > 0:
        img = img.astype(np.int16)
        img += np.random.randint(low=-intensity_noise, high=intensity_noise,
                                 size=img.shape, dtype=np.int16)
        cv2.normalize(img, img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
        img = img.astype(np.uint8)
        
    # Rotation/translation
    rotation_noise = noisy_value_from_type('rotation')
    M, rotate_size = rotate_bound(img,rotation_noise)
    trans_x, trans_y = noisy_value_from_type('translation'),noisy_value_from_type('translation')
    
    # Transform_mat
    M[0, 2] += trans_x
    M[1, 2] += trans_y
    
    ih, iw = rotate_size
    
    # Warp img and masks
    img = cv2.warpAffine(img, M,(iw, ih))
    masks = [cv2.warpAffine(m,M,(iw,ih)) for m in masks]

    # Adjust gt ellipse rotated rect
    rotated_rect[0] = np.dot(M,np.array([rotated_rect[0][0],rotated_rect[0][1],1]))
    rotated_rect[2] -= rotation_noise
   
    if o_shape!=None:
        oh, ow = o_shape
        if ih <= iw:
            scale = float(oh)/ih
        else:
            scale = float(ow)/iw
            
        # Resize masks and images to desired scale using shorter side
        img = cv2.resize(img, None, fx=scale, fy=scale)
        inter_type = cv2.INTER_AREA if flag == 2 else cv2.INTER_NEAREST
        masks = [cv2.resize(m, None, fx=scale, fy=scale,interpolation=inter_type) for m in masks]
        
        # Resize rotated rect 
        rotated_rect[0] = rotated_rect[0]*scale
        rotated_rect[1] = np.array(rotated_rect[1])*scale

        # Center crop to fix width/height ratio
        dy,dx = (img.shape[0] - o_shape[0])//2 ,(img.shape[1] - o_shape[1])//2
        rotated_rect[0] = rotated_rect[0] - [dx,dy]
        
        img = center_crop(img, o_shape)
        masks = [center_crop(m,o_shape) for m in masks]
    
    # Conversion between augmented and cropped
    rect_trans = np.array([scale,dx,dy])
    img = np.transpose(img, (2,0,1))
    img = img.astype(np.float32)
    img *= 2.0 / 255.0
    img -= 1

    if flag == 1:
        # Convert to one-hot target matrix
        # RGB color template {Background, Iris, Sclera}
        color_template = np.array([[49,27,146],[255,238,88],[102,187,106]])
        gt_img = cv2.cvtColor(masks[0],cv2.COLOR_BGR2RGB) 
        d = np.dstack([np.linalg.norm(gt_img - color_template[i],axis=2) for i in range(3)])
        target = np.argmin(d,axis=2) # Returns the indices of the minimum values along an axis
        one_hot = np.zeros(shape=(d.shape)) # 0->[1,0,0] 1->[0,1,0] 2->[0,0,1]
        for i in range(3):
            rows,cols = np.where(target==i)
            one_hot[rows,cols,i] = 1
        
        return img, target, one_hot, rotated_rect, rect_trans

In [ ]:
def raw(img, o_shape, diff):
    """ Crop raw img to desire shape for inference
    KeyArgs:
        oshape(tuple(int,int)): desired image resolution
        diff(float): noise level
    """
    ih, iw = img.shape[0], img.shape[1]

    # Augmentation helper values and methods
    random_multipliers = [] # Noise value need to multiply gaussian
    def value_from_type(augmentation_type):
        # Scale to be in range
        easy_value, hard_value = augmentation_ranges[augmentation_type]
        value = (hard_value - easy_value) * diff + easy_value
        value = (np.clip(value, easy_value, hard_value)
                 if easy_value < hard_value
                 else np.clip(value, hard_value, easy_value))
        return value

    def noisy_value_from_type(augmentation_type):
        # Get normal distributed random value
        if len(random_multipliers) == 0:
            random_multipliers.extend(
                    list(np.random.normal(size=(len(augmentation_ranges),))))
        return random_multipliers.pop() * value_from_type(augmentation_type)
    
    # Bluring images using different types of bluring mehtod
    img = blur_img(img, np.random.choice(blur_type),int(np.abs(noisy_value_from_type('blur'))))
    
    # Flip and intensity
    flip = np.random.choice([True, False]) if diff > 0 else False
    if flip:
        img = cv2.flip(img, 1)
        masks = [cv2.flip(m,1) for m in masks]
        rotated_rect[0][0] = iw - rotated_rect[0][0]
        rotated_rect[2] = rotated_rect[2]*-1
        
    intensity_noise = int(value_from_type('intensity'))
    if intensity_noise > 0:
        img = img.astype(np.int16)
        img += np.random.randint(low=-intensity_noise, high=intensity_noise,
                                 size=img.shape, dtype=np.int16)
        cv2.normalize(img, img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
        img = img.astype(np.uint8)
        
    # Rotation first
    rotation_noise = noisy_value_from_type('rotation')
    M, rotate_size = rotate_bound(img,rotation_noise)
    
    # Translation x&y
    trans_x, trans_y = noisy_value_from_type('translation'),noisy_value_from_type('translation')
    
    # Transform_mat
    M[0, 2] += trans_x
    M[1, 2] += trans_y
    
    ih, iw = rotate_size
    
    # Warp img
    img = cv2.warpAffine(img, M,(iw, ih))
    
    if o_shape!=None:
        oh, ow = o_shape
        if ih <= iw:
            scale = float(oh)/ih
        else:
            scale = float(ow)/iw
            
        # Resize and images to desired scale using shorter side
        img = cv2.resize(img, None, fx=scale, fy=scale)
        # Center crop to fix width/height ratio
        dy,dx = (img.shape[0] - o_shape[0])//2 ,(img.shape[1] - o_shape[1])//2
        img = center_crop(img, o_shape)

    img = np.transpose(img, (2,0,1))
    img = img.astype(np.float32)
    img *= 2.0 / 255.0
    img -= 1
    
    rect_trans = np.array([scale,dx,dy])
    return img,rect_trans